In [21]:
import json
import pickle as pkl
import pandas as pd
# !python -m pip install --upgrade pip
# !python -m pip install spacy
import spacy
import csv
import re as re
from tqdm import tqdm
from spacy.util import filter_spans
from spacy.matcher import Matcher
import re
# !python -m spacy download en_core_web_sm
# !python -m spacy download en
from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("english")
nlp = spacy.load(r'C:\Users\gunda\AppData\Local\Programs\Python\Python38\Lib\site-packages\spacy\data\en\en_core_web_sm-2.3.1')

In [2]:
def clean(text):
    return re.sub(r'[^a-zA-Z ]', "", text).lower().lstrip().rstrip()

In [3]:
def get_sentences(texts):
    print("get_sentences function is running")
    docs = list(nlp.pipe(texts, disable=['ner']))
    sentences = []
    for doc in tqdm(docs):
        list_sent = list(doc.sents)
        for sent in list_sent:
            sentences.append(sent.text)
    return sentences

In [4]:
def get_pos_ent(text):
    
    doc = nlp(text)
    noun_chuncks = list(doc.noun_chunks)
    np = [noun_chunck.text for noun_chunck in noun_chuncks]
    np = remove_stop_words(np)
    n = [tok.text for tok in doc if tok.pos_ in ['NOUN', 'PROPN'] and tok.is_stop== False]
    n = remove_stop_words(n)
    ent = [ent.text for ent in doc.ents]
    ent = remove_stop_words(ent)
    final = np + n + ent
    final = list(set(final))
    features = {'ne_np':final, 'sentence':text}
    return features

In [6]:
np = ['Someone', 'my attention', 'these 2 letters', 'the Daily Mail', 'I']
def remove_stop_words(np):
    
    stop_words = [line.rstrip('\n') for line in open('stopwordList.txt')]
    new_np = []
    for n in np: 
        words = n.split()
        words = [w for w in words if w.lower() not in stop_words]
        word_to_add = ' '.join(words)
        if len(word_to_add) > 0:
            word_to_add = clean(word_to_add)
            new_np.append(str(word_to_add))
    return new_np
remove_stop_words(np)

['attention', 'letters', 'daily mail']

In [7]:
data = pd.read_csv('YInt.csv')
content =  data['message'].values.tolist()
content = [str(message) for message in content]
print("Total number of messages: ",len(content))
content_b1 = content[:10485]
content_b2 = content[10485:20970]
content_b3 = content[20970:31455]
content_b4 = content[31455:]

Total number of messages:  41941


data = pd.read_csv('YInt.csv')
content =  data['message'].values.tolist()
content = [str() for blog in blog_content]
blog_content_b1 = blog_content[:7000]
blog_content_b2 = blog_content[7000:15000]
blog_content_b3 = blog_content[15000:22000]
blog_content_b4 = blog_content[22000:]
'''

In [8]:
# sentences_b1 = get_sentences(content_b1)
sentences_b1 = content_b1
pkl.dump(sentences_b1, open('temp/sentences_b1.pkl','wb'))

In [9]:
# sentences_b2 = get_sentences(content_b2)
sentences_b2 = content_b2
pkl.dump(sentences_b2, open('temp/sentences_b2.pkl','wb'))

In [10]:
# sentences_b3 = get_sentences(content_b3)
sentences_b3 = content_b3
pkl.dump(sentences_b3, open('temp/sentences_b3.pkl','wb'))

In [11]:
# sentences_b4 = get_sentences(content_b4)
sentences_b4 = content_b4
pkl.dump(sentences_b4, open('temp/sentences_b4.pkl','wb'))

In [12]:
sentences_b1 = pkl.load(open('temp/sentences_b1.pkl','rb'))
sentences_b2 = pkl.load(open('temp/sentences_b2.pkl','rb'))
sentences_b3 = pkl.load(open('temp/sentences_b3.pkl','rb'))
sentences_b4 = pkl.load(open('temp/sentences_b4.pkl','rb'))
sentences_all = sentences_b1 + sentences_b2 + sentences_b3 + sentences_b4
len_sentences_all = len(sentences_b1) + len(sentences_b2) + len(sentences_b3) + len(sentences_b4)
print(len_sentences_all)
print(len(sentences_all))
pkl.dump(sentences_all, open('temp/sentences_all.pkl','wb'))

41941
41941


In [13]:
sentences_all = pkl.load(open('temp/sentences_all.pkl','rb'))
sentences_all_chuncks = [sentences_all[i:i+10000] for i in range(0, len(sentences_all), 10000)]

In [14]:
for i, sentences_all_chunck in enumerate(tqdm(sentences_all_chuncks)):
    print(i)
    features = []
    for text in sentences_all_chunck:
        feature = get_pos_ent(text)
        features.append(feature)
    file_name = './temp/features_chunck_'+str(i)+'.pkl'
    pkl.dump(features, open(file_name,'wb'))

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

0


 20%|████████████████▊                                                                   | 1/5 [01:17<05:11, 77.90s/it]

1


 40%|█████████████████████████████████▌                                                  | 2/5 [02:49<04:06, 82.07s/it]

...

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [04:27<02:53, 86.94s/it]

3


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [06:15<01:33, 93.04s/it]

4


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [06:34<00:00, 78.85s/it]


In [15]:
ne_np_sent_all = []
for i in range(len(sentences_all_chuncks)):
    file_name = './temp/features_chunck_'+str(i)+'.pkl'
    tmp = pkl.load(open(file_name, 'rb'))
#     print(len(tmp))
    ne_np_sent_all.extend(tmp)
pkl.dump(ne_np_sent_all, open('./temp/ne_np_sent_all.pkl','wb'))
print(len(ne_np_sent_all))

41941


In [16]:
ne_np_frame = pd.DataFrame(ne_np_sent_all, columns = ['ne_np' , 'sentence'])
#ne_np_frame.to_csv('features_of_messages.csv', index = False)

In [17]:
data['features']= [",".join(val) for val in ne_np_frame['ne_np'].values.tolist()]
data.to_csv('new_YInt.csv', index = False)

In [19]:
## Please run counter-Copy1 and code the feature labels before running the below code

features = pd.read_excel('new_YInt_NE_NP_coded.xlsx')
print(features.count())
print(features.head())

ne_np     621
count     621
coding    621
dtype: int64
       ne_np  count           coding
0     anyone   3694  May be Required
1    someone   3529  May be Required
2   anything   3284  May be Required
3  something   3173  May be Required
4     people   1584  May be Required


In [20]:
coding = ['May be Required','Important','Unimportant']

In [21]:
features_dict = {}
for index, row in features.iterrows():
    features_dict[row['ne_np']] = row['coding']
print(len(features_dict.keys()))
data = pd.read_csv('new_YInt.csv')

621


In [22]:
#df = pd.DataFrame(columns=data.columns)
dropIndexes = []
for i,j in tqdm(data.iterrows()):
    row = list(j)
    features = row[4]
    removeRow = True
    if type(features)==type(1.00):
        continue
    for feature in features.split():
        if feature not in features_dict.keys():
            removeRow = False
            break
        if features_dict[feature]!=coding[2]:
            print(features_dict[feature])
            removeRow = False
            break
    if removeRow:
        print(i)
        dropIndexes.append(i)

1781it [00:00, 17648.86it/s]

Important
14
21
Important
72
May be Required
118
125
151
May be Required
168
Important
183
May be Required
Important
May be Required
293
May be Required
May be Required
349
393
461
462
481
May be Required
513
Important
561
572
May be Required
May be Required
May be Required
595
603
609
615
635
641
656
665
666
671
684
685
687
692
696
700
706
707
717
723
725
728
734
746
Important
May be Required
755
768
769
May be Required
783
786
May be Required
May be Required
796
May be Required
811
820
823
829
844
Important
861
863
Important
Important
Important
May be Required
893
Important
897
900
May be Required
Important
Important
924
931
940
May be Required
942
948
Important
956
966
979
May be Required
Important
983
989
Important
994
1010
1015
1018
1021
Important
1024
1036
1051
1061
Important
1072
1091
1097
1107
Important
Important
1120
1124
Important
1130
1142
Important
1159
Important
1180
May be Required
1208
1218
Important
May be Required
1256
1271
1280
1283
1289
1290
May be Required
1309
1319

7208it [00:00, 17884.25it/s]


3285
Important
3325
Important
3363
3373
3376
May be Required
3400
3405
3418
3445
3459
May be Required
3465
3481
May be Required
3493
3500
3512
3516
Important
May be Required
3548
3559
3566
3573
Important
3625
3627
Important
May be Required
3654
3659
May be Required
3677
Important
Important
Important
Important
3708
Important
Important
3757
Important
Important
3784
3785
Important
Important
3797
Important
May be Required
Important
May be Required
Important
Important
May be Required
May be Required
Important
3862
May be Required
Important
Important
May be Required
May be Required
May be Required
May be Required
Important
Important
May be Required
May be Required
Important
May be Required
May be Required
4071
May be Required
4087
4101
4134
May be Required
4144
May be Required
4174
Important
4199
4223
4229
4233
May be Required
May be Required
Important
Important
4286
4287
4288
Important
4313
Important
4328
Important
May be Required
4375
Important
Important
May be Required
Important
4401
443

10773it [00:00, 17526.36it/s]

...


31550
Important
31562
Important
31575
Important
Important
Important
31650
Important
31680
Important
31695
Important
Important
Important
31716
Important
Important
Important
Important
31747
Important
Important
Important
Important
Important
Important
31787
Important
Important
Important
Important
Important
May be Required
Important
Important
Important
Important
Important
Important
Important
Important
Important
Important
Important
Important
Important
Important
Important
31905
Important
May be Required
Important
Important
Important
Important
Important
Important
Important
32024
Important
Important
Important
Important
Important
32112
Important
32127
32138
32151
32153
32169
Important
32178
32186
May be Required
Important
32201
32205
32214
32225
Important
32237
Important
Important
32250
May be Required
32262
May be Required
32283
32288
Important
32304
May be Required
May be Required
May be Required
32352
May be Required
May be Required
Important
May be Required
May be Required
Important
May be 

37798it [00:02, 16482.76it/s]


Important
35353
Important
May be Required
May be Required
May be Required
Important
May be Required
Important
35432
Important
Important
35491
35498
Important
35512
35514
Important
Important
35545
May be Required
May be Required
May be Required
Important
Important
May be Required
May be Required
Important
35615
35622
Important
35632
Important
Important
Important
35681
35698
35710
Important
May be Required
35732
Important
Important
Important
35753
Important
May be Required
35791
35800
Important
35803
Important
May be Required
Important
Important
35832
35838
Important
35861
35870
May be Required
May be Required
35903
35904
35934
May be Required
Important
35947
Important
35960
35968
Important
Important
Important
36003
36004
36006
Important
36030
36031
Important
36049
36065
36072
36080
36092
Important
36095
Important
36115
Important
36144
May be Required
36184
Important
36212
Important
Important
36248
Important
Important
36280
36281
Important
Important
Important
36319
36324
Important
36331

41941it [00:02, 17382.92it/s]


38165
38168
Important
38174
Important
38182
38184
Important
38191
Important
38200
38202
38227
Important
38237
Important
Important
Important
38258
Important
May be Required
Important
May be Required
38339
38345
Important
Important
Important
Important
Important
Important
Important
38370
Important
Important
38376
38377
38379
38380
Important
38382
Important
38384
38385
Important
38393
Important
Important
Important
Important
Important
Important
Important
Important
Important
38412
Important
Important
Important
Important
Important
Important
Important
Important
38436
Important
38448
Important
Important
38459
38461
Important
Important
38477
Important
May be Required
38481
Important
38487
38493
Important
38504
Important
Important
Important
Important
38591
Important
38623
38625
38631
38659
May be Required
38671
Important
38677
Important
May be Required
38734
May be Required
38739
Important
Important
May be Required
Important
May be Required
Important
Important
38809
May be Required
Important
Imp

In [23]:
data.drop(dropIndexes,inplace=True)

In [24]:
print(len(dropIndexes))
data.count()
data.to_csv('new_YInt_deleted_allUnimportant.csv', index = False)

1841


In [55]:
data = pd.read_csv('new_YInt_deleted_allUnimportant.csv')

In [56]:
delete_accounts = ["Syndicated4", "Syndicated5", "Syndicated348","______3333_____", "J0rdanWantsBacon", "JordanWantsBac0n", "CantonCoordon2",
"Opportunities2", "Opportunities1"]
dropIndexes = []
for i,j in tqdm(data.iterrows()):
    row = list(j)
    if row[2] in delete_accounts:
        dropIndexes.append(i)

40100it [00:01, 21351.37it/s]


In [57]:
data.drop(dropIndexes,inplace=True)
data.to_csv('new_YInt_deleted_adv_accounts.csv', index = False)

In [28]:
data = pd.read_csv('new_YInt_deleted_adv_accounts.csv')

In [29]:
imp_accounts = ["StHimarkEMA","DOT-StHimark","StHimark","Survey-SHM","SHM-WaterSewer","CityHealth-SMH","KRAKTV","AlwaysSafePowerCompany",\
"SHM-PowerC","ThesaurusGasCompany","SHM-SaveAnimals","StHimark-Learns","Sal","Nicky77","WhatNowHow","theaterlover34",\
"kyle","ElwinJR","Hiker4Life","AlwaysUpToEat34","You2","Runner23","TakeABreak","FrancineHiggins","ICYou","Rescue14",\
"Mommy2Four","FireCaptainJack","Doood","Rescue12","CityEvents","LivesToHike79","firefighter87","SunnyDay",\
"RunningTree","Mrs.Tillenbottom","Bopper","LoopyLou","HurtsToPurr","IfIOnlyHadATrain","Booper","DoNotWant",\
"OldTownGirl","MyNameIsYourName2","BBGirl","RoseWyatt","BoogieBunny","HimarkGrocersAssc","Olivia","EllaChang",\
"Bonanza4Evr","RhyminSimonJBread","BarryB","CiloFlame","GreenTMouse","StHimarkMusicFest","Milo99","MindyLevy",\
"ChapparelFD","TommyNotThatTommy","Oppsie","wftrubble","Bonanze4Evr","DareYouToPetTheCat","Westie","RayOfRain",\
"localburger","HopeSpringsEternal","ghosty","OtherJoey","BroadviewBoi","ArtistLalisaManoban","kristen","BoringJoe",\
"ComeBackJack","retiredfiremarshall","ImNotUR","Phone4Life","SlimRodents2","AlwaysSafePowerCompany","Chappie",\
"DerekNolan","biscuitJtoot","StHimarkMutualInsuranceCo","ElwinJr","NoFearCher","VanessaCorwin","UnitedChurchOfJohnsonville",\
"CityEOC","WalkerTree","LivesToHike79","EmielRegis","BB","Girl","RockyFlame","2NDammendment","FireCpn"]
impIndexes = []
for i,j in tqdm(data.iterrows()):
    row = list(j)
    if row[2] in imp_accounts:
        impIndexes.append(i)
    else:
        if type(row[3])==type(1.00):
            continue
        for account in imp_accounts:
            if account in row[3]:
                impIndexes.append(i)
                break

39390it [00:02, 15616.33it/s]


In [30]:
len(impIndexes)

2111

In [31]:
for i in impIndexes:
    #print(data.loc[i]['message'])
    pass

In [32]:
data.loc[impIndexes].to_csv('new_YInt_imp_accounts.csv', index = False)


In [23]:
data = pd.read_csv('new_YInt_deleted_adv_accounts.csv')
earthquakeList = [englishStemmer.stem(string) for string in ['temblor', 'wobbl', 'quiver', 'trumbl', 'tremor', 'quak', 'seismic', 'shake', 'epicent', 'earthquak', 'shook']]
sewer_and_water= [englishStemmer.stem(string) for string in ["discharged", "discharge", "drain", "drainage", "flood", "hygiene", "irrigation", "pipes", "pump", "river", "sanitary", "sewage", "sewer", "stream", "underground", "wash", "waste", "water"]]
power_energy= [englishStemmer.stem(string) for string in ["valve", "heat", "gas", "power", "electric", "candle", "flashlight", "generator", "black out", "blackout", "dark", "radiation", "radio rays", "energy", "nuclear", "fuel", "battery", "radiant"]]
roads_and_bridges= [englishStemmer.stem(string) for string in ["airport", "avenue", "bridge", "bus", "congestion", "drive", "flight", "jam", "logistic", "metro", "mta", "road", "street", "subway", "traffic", "train", "transit", "transportation", "highway", "route", "lane"]]
medical= [englishStemmer.stem(string) for string in ["medical", "red cross", "food", "emergency", "urgent", "evacuate", "evacuating", "evacuation", "protection", "ambulance", "escape", "first aid", "rescue", "rescuing", "dead", "death", "kill", "help", "help out", "help with", "volunteer", "volunteering", "explosion", "exploding", "explode", "victim", "fatalities"]]
buildings=[englishStemmer.stem(string) for string in ["collapse", "housing", "house"]]
stop_words = [line.rstrip('\n') for line in open('stopwordList.txt')]
for i,j in tqdm(data.iterrows()):
    row = list(j)
    words = []
    resource_column = False
    if not pd.isnull(data.loc[i,'message']):
        for string in re.split(' |;|,|!|#|@|$|&|\?|\.|\'|\*',row[3]):
            string = re.sub(r'\W+', '',string)
            if string and string.lower() not in stop_words:
                words.append(string)
            if englishStemmer.stem(string).lower() in earthquakeList+sewer_and_water+power_energy+roads_and_bridges+medical\
            +buildings:
                resource_column = True
    if len(words)==0:
        data.loc[i,'formatted_message'] = None
    elif len(words)==1:
        data.loc[i,'formatted_message'] = words[0]
    else:
        data.loc[i,'formatted_message']= ",".join(words)
    if resource_column:
        data.loc[i,'resource_present']= 1
    else:
        data.loc[i,'resource_present']= 0

39390it [00:48, 807.15it/s]


In [24]:
#data = pd.read_csv('new_YInt_deleted_adv_accounts.csv')
#data["new_time"]=pd.to_datetime(data["time"]).dt.strftime('%Y-%m-%d %H')
data.to_csv('new_YInt_deleted_adv_accounts.csv', index = False)